# Segmenting and Clustering Neighborhoods in Toronto

**This Notebook will explore and cluster the neighborhoods in Toronto**

**Scraping data from Wikipedia Page**


In [1]:
import requests 
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

In [2]:
My_table = soup.find('table',{'class':'wikitable sortable'})


In [3]:
Postcode = []
Borough = []
Neighbourhood = []

for row in My_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        Postcode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighbourhood.append(cells[2].find(text=True))

In [4]:
import pandas as pd
df=pd.DataFrame()
df['PostalCode']=Postcode
df['Borough']=Borough
df['Neighbourhood']=Neighbourhood
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
df1 = df[df.Borough != 'Not assigned']
df1.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [6]:
df2 = df1.groupby("PostalCode").agg(lambda x:','.join(set(x)))
df2.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill"
M1E,Scarborough,"West Hill,Morningside,Guildwood\n"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae\n


In [7]:
df2.shape

(103, 2)

**Importing Geospatial data into dataframe (data)**

In [8]:
data = pd.read_csv('http://cocl.us/Geospatial_data', delimiter = ',')
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
data_merge = pd.merge(left=df2,right=data, left_on='PostalCode', right_on='Postal Code')
data_merge.head()

,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,Scarborough,"Malvern,Rouge",M1B,43.806686,-79.194353
1,Scarborough,"Port Union,Highland Creek,Rouge Hill",M1C,43.784535,-79.160497
2,Scarborough,"West Hill,Morningside,Guildwood\n",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae\n,M1H,43.773136,-79.239476


In [10]:
data_merge = data_merge[['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
data_merge.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"West Hill,Morningside,Guildwood\n",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476


**Segmenting Borough that contains the word Toronto**

In [11]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#conda install -c conda-forge folium 
import folium

In [12]:
data_toronto = data_merge[data_merge['Borough'].str.contains('Toronto')].reset_index(drop=True)
data_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale,The Danforth West\n",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar,The Beaches West\n",43.668999,-79.315572
3,M4M,East Toronto,Studio District\n,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [13]:
import numpy as np
toronto_map = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = data_toronto['Latitude']
Y = data_toronto['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
data_toronto['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(data_toronto['Latitude'], data_toronto['Longitude'], data_toronto['Borough'], data_toronto['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_map)  

toronto_map

**FOURSQUARE VENUES**

In [14]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass
                 
                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [15]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'RHJWADLSSXF5XOAMYFV5LSAENFKW3GDQYACOUDHA151K5DYX'
CLIENT_SECRET = 'PNPGD3KJVGLBI3TDOZI4GRCMCRJG1YMJS4BXLDL25HV0N4SW'
VERSION = '20180604'

In [18]:
toronto_venues_indian = getNearbyVenues(names=data_toronto['Neighbourhood'], latitudes=data_toronto['Latitude'], longitudes=data_toronto['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d10f941735')
toronto_venues_indian.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Delhi Bistro,43.672506,-79.288353,Indian Restaurant
1,The Beaches,43.676357,-79.293031,ali's tandoori curry house,43.673393,-79.283740,Indian Restaurant
2,The Beaches,43.676357,-79.293031,Chef of India,43.672100,-79.304773,Indian Restaurant
3,"Riverdale,The Danforth West\n",43.679557,-79.352188,Sher-E-Punjab,43.677308,-79.353066,Indian Restaurant
4,"Riverdale,The Danforth West\n",43.679557,-79.352188,Namaste Nepal,43.678635,-79.346626,Indian Restaurant


In [19]:
toronto_venues_indian.shape

(529, 7)

In [20]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [21]:
map_toronto_indian = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(toronto_venues_indian, 'red', map_toronto_indian)

map_toronto_indian


In [22]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [23]:
toronto_grouped = toronto_venues_indian.groupby('Neighborhood').count()
toronto_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide\n,Richmond\n,King\n",42,42,42,42,42,42
Berczy Park,18,18,18,18,18,18
"Brockton\n,Parkdale Village,Exhibition Place",9,9,9,9,9,9
Business Reply Mail Processing Centre 969 Eastern\n,10,10,10,10,10,10
"Cabbagetown,St. James Town",8,8,8,8,8,8
Central Bay Street\n,33,33,33,33,33,33
Christie\n,8,8,8,8,8,8
Church and Wellesley,34,34,34,34,34,34
"Commerce Court,Victoria Hotel\n",41,41,41,41,41,41


**Analysis of Each Neighborhood**

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_indian[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_indian['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Asian Restaurant,Burger Joint,Caribbean Restaurant,Fast Food Restaurant,Food Truck,Gastropub,Halal Restaurant,Indian Chinese Restaurant,Indian Restaurant,Italian Restaurant,Kebab Restaurant,North Indian Restaurant,Snack Place,Soup Place,Sri Lankan Restaurant
0,The Beaches,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,"Riverdale,The Danforth West\n",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,"Riverdale,The Danforth West\n",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Asian Restaurant,Burger Joint,Caribbean Restaurant,Fast Food Restaurant,Food Truck,Gastropub,Halal Restaurant,Indian Chinese Restaurant,Indian Restaurant,Italian Restaurant,Kebab Restaurant,North Indian Restaurant,Snack Place,Soup Place,Sri Lankan Restaurant
0,"Adelaide\n,Richmond\n,King\n",0.000000,0.000000,0.000000,0.0,0.023810,0.000000,0.000000,0.000000,0.952381,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810
1,Berczy Park,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.944444,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556
2,"Brockton\n,Parkdale Village,Exhibition Place",0.000000,0.000000,0.111111,0.0,0.000000,0.000000,0.000000,0.000000,0.777778,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 East...,0.000000,0.100000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.800000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000
4,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.875000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000
5,Central Bay Street\n,0.000000,0.000000,0.000000,0.0,0.030303,0.000000,0.000000,0.000000,0.939394,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303
6,Christie\n,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.000000,0.000000,0.029412,0.0,0.029412,0.000000,0.000000,0.000000,0.911765,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000
8,"Commerce Court,Victoria Hotel\n",0.000000,0.000000,0.000000,0.0,0.024390,0.000000,0.000000,0.000000,0.951220,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390
9,Davisville\n,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide\n,Richmond\n,King\n",Indian Restaurant,Sri Lankan Restaurant,Food Truck,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant
1,Berczy Park,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
2,"Brockton\n,Parkdale Village,Exhibition Place",Indian Restaurant,North Indian Restaurant,Caribbean Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant
3,Business Reply Mail Processing Centre 969 East...,Indian Restaurant,Snack Place,Burger Joint,Sri Lankan Restaurant,Soup Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant
4,"Cabbagetown,St. James Town",Indian Restaurant,Kebab Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub


In [29]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 3, 3, 0, 0, 4, 0, 0, 4], dtype=int32)

In [69]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = data_toronto
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
1,M4K,East Toronto,"Riverdale,The Danforth West\n",43.679557,-79.352188,0,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
2,M4L,East Toronto,"India Bazaar,The Beaches West\n",43.668999,-79.315572,0,3.0,Indian Restaurant,Snack Place,Indian Chinese Restaurant,Halal Restaurant,Gastropub,Burger Joint,Sri Lankan Restaurant,Soup Place,North Indian Restaurant,Kebab Restaurant
3,M4M,East Toronto,Studio District\n,43.659526,-79.340923,0,1.0,Indian Restaurant,Fast Food Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,M4P,Central Toronto,Davisville North\n,43.712751,-79.390197,2,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
6,M4R,Central Toronto,North Toronto West\n,43.715383,-79.405678,2,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
7,M4S,Central Toronto,Davisville\n,43.704324,-79.388790,2,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
8,M4T,Central Toronto,"Moore Park,Summerhill East\n",43.689574,-79.383160,2,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
9,M4V,Central Toronto,"Summerhill West\n,Deer Park,South Hill,Forest ...",43.686412,-79.400049,2,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub


In [72]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Downtown Toronto,1,0.0,Indian Restaurant,Kebab Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
12,Downtown Toronto,1,0.0,Indian Restaurant,Soup Place,Food Truck,Caribbean Restaurant,Sri Lankan Restaurant,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant
13,Downtown Toronto,1,0.0,Indian Restaurant,Kebab Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
14,Downtown Toronto,1,0.0,Indian Restaurant,Sri Lankan Restaurant,Food Truck,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant
15,Downtown Toronto,1,0.0,Indian Restaurant,Sri Lankan Restaurant,Kebab Restaurant,Food Truck,Soup Place,Snack Place,North Indian Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant
16,Downtown Toronto,1,0.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
17,Downtown Toronto,1,0.0,Indian Restaurant,Sri Lankan Restaurant,Food Truck,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant
18,Downtown Toronto,1,0.0,Indian Restaurant,Sri Lankan Restaurant,Food Truck,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant
19,Downtown Toronto,1,0.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
20,Downtown Toronto,1,0.0,Indian Restaurant,Sri Lankan Restaurant,Food Truck,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant


In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,0,1.0,Indian Restaurant,Fast Food Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant


In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,West Toronto,3,2.0,Indian Restaurant,Italian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub


In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,0,3.0,Indian Restaurant,Snack Place,Indian Chinese Restaurant,Halal Restaurant,Gastropub,Burger Joint,Sri Lankan Restaurant,Soup Place,North Indian Restaurant,Kebab Restaurant
33,West Toronto,3,3.0,Indian Restaurant,North Indian Restaurant,Caribbean Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant
37,East Toronto,0,3.0,Indian Restaurant,Snack Place,Burger Joint,Sri Lankan Restaurant,Soup Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant


In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
1,East Toronto,0,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
5,Central Toronto,2,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
6,Central Toronto,2,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
7,Central Toronto,2,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
8,Central Toronto,2,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
9,Central Toronto,2,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
10,Downtown Toronto,1,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
22,Central Toronto,2,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
23,Central Toronto,2,4.0,Indian Restaurant,Sri Lankan Restaurant,Soup Place,Snack Place,North Indian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Gastropub
